# Grading curve

This course has a curve, per [SIPA policy](https://bulletin.columbia.edu/sipa/academic-policies/satisfactory-academic-progress/):

> Grades submitted for SIPA core courses must have an average GPA between 3.2 and 3.4, with the goal being 3.3.

---

## How course grades work

1. The grades are computed using [the weights listed in the syllabus](../README.md#grading).
1. The percentage required for all the letter grades is adjusted up or down as necessary to hit the target GPA range across both sections.
1. The letter grades are [uploaded to SSOL](https://registrar.columbia.edu/content/managing-grades).

The `min_score` column of the [new cutoffs](#new-cutoffs) shows the current minimum `Total` percentage required for each letter grade.

## Methodology

The rest of this notebook shows how the grade cutoffs are computed. <!--**This methodology, the distribution of student percentages, and thus your estimated course grade are subject to change** up until final grades are submitted.-->


In [1]:
MIN_AVG_GPA = 3.2
MAX_AVG_GPA = 3.4
NUM_STUDENTS = 15

### Load current scores


In [2]:
import pandas as pd

grades = pd.read_csv(
    "~/Downloads/2025-05-11T2035_Grades-INAFU6659_001_2025_1_-_Advanced_Computing_for_Policy.csv",
    skiprows=[1, 2],
)
grades = grades[grades["Student"] != "Student, Test"]
assert grades.shape[0] == NUM_STUDENTS
grades = grades.sort_values("Final Score")

#### Distribution


In [3]:
import plotly.express as px

fig = px.histogram(
    grades,
    x="Final Score",
    title="Distribution of the overall grades",
    labels={"Final Score": "Final Score (percent)"},
)
fig.update_layout(yaxis_title_text="Number of students")
fig.show()

### Match to letter grades / GPAs

Creating the [grading notation table](https://bulletin.columbia.edu/sipa/academic-policies/satisfactory-academic-progress/) in Pandas:


In [4]:
letter_grade_equivalents = pd.DataFrame(
    index=["A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D", "F"],
    data={"gpa": [4.00, 3.67, 3.33, 3.00, 2.67, 2.33, 2.00, 1.67, 1.00, 0.00]},
)

Assign starting minimum scores, roughly based on the Default Canvas Grading Scheme:


In [5]:
letter_grade_equivalents["min_score"] = [
    94.0,
    90.0,
    87.0,
    84.0,
    80.0,
    77.0,
    74.0,
    70.0,
    60.0,
    0.0,
]
letter_grade_equivalents

,gpa,min_score
A,4.00,94.0
A-,3.67,90.0
B+,3.33,87.0
B,3.00,84.0
B-,2.67,80.0
C+,2.33,77.0
C,2.00,74.0
C-,1.67,70.0
D,1.00,60.0
F,0.00,0.0


### Adjust cutoffs

Raise or lower the minimum scores for each grade (not including F) until the average GPA is in the acceptable range.


In [6]:
# merge_asof() needs columns sorted ascending
orig_grade_cutoffs = letter_grade_equivalents.sort_values(by="min_score")
grade_cutoffs = orig_grade_cutoffs.copy()

grades_to_adjust = grade_cutoffs.index != "F"

adjustment = 0
STEP_SIZE = 0.1

while True:
    grade_cutoffs.loc[grades_to_adjust, "min_score"] = (
        orig_grade_cutoffs[grades_to_adjust]["min_score"] + adjustment
    )

    # make the letter grades a column so they show up in the merged DataFrame
    grade_cutoffs_with_letters = grade_cutoffs.reset_index().rename(
        columns={"index": "letter_grade"}
    )

    # find the letter grade / GPA for each student
    adjusted_grades = pd.merge_asof(
        grades,
        grade_cutoffs_with_letters,
        left_on="Final Score",
        right_on="min_score",
        direction="backward",
    )

    new_mean = adjusted_grades["gpa"].mean()
    print(f"Adjustment: {adjustment:+.1f}, Average: {new_mean:.3f}")

    # check if we've hit the target range
    if MIN_AVG_GPA <= new_mean < MAX_AVG_GPA:
        # success
        break
    elif new_mean >= MAX_AVG_GPA:
        # raise
        adjustment += STEP_SIZE
    else:  # new_mean < MIN_AVG_GPA:
        # lower
        adjustment -= STEP_SIZE

Adjustment: +0.0, Average: 3.245


Confirm the A cutoff is still achievable:


In [7]:
assert grade_cutoffs.at["A", "min_score"] <= 100

#### New cutoffs


In [8]:
grade_cutoffs.sort_values("min_score", ascending=False)

,gpa,min_score
A,4.00,94.0
A-,3.67,90.0
B+,3.33,87.0
B,3.00,84.0
B-,2.67,80.0
C+,2.33,77.0
C,2.00,74.0
C-,1.67,70.0
D,1.00,60.0
F,0.00,0.0


### Check results


Double-check the new average is in line with policy:


In [9]:
assert MIN_AVG_GPA <= new_mean < MAX_AVG_GPA, f"{new_mean} not in acceptable range"

new_mean

np.float64(3.244666666666667)

In [10]:
fig = px.histogram(
    adjusted_grades, x="letter_grade", title="Distribution of letter grades"
)
fig.update_layout(yaxis_title_text="Number of students")
fig.show()

## Export

Format needed by SSOL.


In [11]:
import os

ssol = adjusted_grades.rename(columns={"SIS User ID": "UNI", "letter_grade": "grade"})
ssol = ssol[["UNI", "grade"]].sort_values("UNI")

os.makedirs("../tmp", exist_ok=True)
ssol.to_csv("../tmp/grades.csv", index=False)
print("Done")

Done
